<a href="https://colab.research.google.com/github/skabra5/Streaming-tweets-with-Tweepy-Visualing-tweets-using-Plotly-real-time-topic-tracking-with-Dash/blob/master/DashApp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/TwitterDash")

In [1]:
!pip install dash dash-renderer dash-html-components dash-core-components plotly

     |████████████████████████████████| 71kB 3.0MB/s 
     |████████████████████████████████| 1.2MB 7.9MB/s 
     |████████████████████████████████| 194kB 16.0MB/s 
     |████████████████████████████████| 3.4MB 20.1MB/s 
     |████████████████████████████████| 1.8MB 45.5MB/s 
     |████████████████████████████████| 358kB 43.4MB/s 
  Created wheel for dash: filename=dash-1.12.0-cp36-none-any.whl size=73892 sha256=b5341f686e885f717bbc9b32c740273706b0f20cfd95e7eb5c5aa46b010972f2
  Stored in directory: /root/.cache/pip/wheels/2e/bc/dd/b4ebecb434c90eff7c5475efc10b72aad5a899c01fa87e2647
  Created wheel for dash-renderer: filename=dash_renderer-1.4.1-cp36-none-any.whl size=1165597 sha256=22490a89fb04d78cc3bcb22fd96bd40a92a833e916aa93e51f5305eaef5bc69a
  Stored in directory: /root/.cache/pip/wheels/be/9e/83/c6531c3fdd6dba27156e7196c273148d850c97bda396fec7d0
  Created wheel for dash-html-components: filename=dash_html_components-1.0.3-cp36-none-any.whl size=427303 sha256=796eaa4780424e92d855682

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
import pandas as pd
import plotly.graph_objs as go
from django.conf import settings
import itertools
import math
import base64
from flask import Flask
import os
import psycopg2
import datetime
import plotly
import random
from collections import deque
 
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import sqlite3



/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


--2020-06-10 07:09:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.208.90.15, 18.214.118.253, 3.223.98.104, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.208.90.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  16.4MB/s    in 0.8s    

2020-06-10 07:09:01 (16.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://db05ac8f461d.ngrok.io


In [7]:
### Save file with Dash app on the Google Colab machine
%%writefile my_app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

my_app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
my_app.scripts.config.serve_locally = True
my_app.title = "Real-time-twitter-monitor"

my_app.layout = html.Div(children=[
    html.H2('Real-time Twitter Sentiment Analysis for Brand Improvement and Topic Tracking ', style={
        'textAlign': 'center'
    }),
    html.H4('(Last updated: June 06, 2020)', style={
        'textAlign': 'right'
    }),
    

    html.Div(id='live-update-graph'),
    html.Div(id='live-update-graph-bottom'),

    # Author's Words
    html.Div(
        className='row',
        children=[ 
            dcc.Markdown("__Author's Words__: I am student of Science and I love wrangling with data!✨"),
        ],style={'width': '35%', 'marginLeft': 70}
    ),
    html.Br(),
    
    # ABOUT ROW
    html.Div(
        className='row',
        children=[
            html.Div(
                className='three columns',
                children=[
                    html.P(
                    'Data extracted from:'
                    ),
                    html.A(
                        'Twitter API',
                        href='https://developer.twitter.com'
                    )                    
                ]
            ),
            html.Div(
                className='three columns',
                children=[
                    html.P(
                    'More cool Projects:'
                    ),
                    html.A(
                        'GitHub',
                        href='https://github.com/skabra5'
                    )                    
                ]
            ),
            html.Div(
                className='three columns',
                children=[
                    html.P(
                    'Made with:'
                    ),
                    html.A(
                        'Dash / Plot.ly',
                        href='https://plot.ly/dash/'
                    )                    
                ]
            ),
            html.Div(
                className='three columns',
                children=[
                    html.P(
                    'Author:'
                    ),
                    html.A(
                        'Sakshi Kabra',
                        href='https://www.linkedin.com/in/sakshi--kabra/'
                    )                    
                ]
            )                                                          
        ], style={'marginLeft': 70, 'fontSize': 16}
    ),

    dcc.Interval(
        id='interval-component-slow',
        interval=1*10000, # in milliseconds
        n_intervals=0
    )
    ], style={'padding': '20px'})


# Multiple components can update everytime interval gets fired.
@my_app.callback(Output(component_id='live-update-graph', component_property='children'),[Input(component_id='interval-component-slow', component_property='n_intervals')])
def update_graph_scatter():
  conn = sqlite3.connect('/content/twitter.db')
  c = conn.cursor()
  df = pd.read_sql_query("SELECT * from sentiment", conn)
  # conn.close()

  # Convert UTC into PDT
  df['unix'] = pd.to_datetime(df['unix'], unit='ms')


  # Clean and transform data to enable time series
  result = df.groupby([pd.Grouper(key='unix', freq='10s'), 'sentiment']).count().unstack(fill_value=0).stack().reset_index()
  result = result.rename(columns={"tweet_id": "Num of tweets", "unix":"Time"})  
  time_series = result["Time"][result['sentiment']==0.0].reset_index(drop=True)

  min10 = datetime.datetime.now() - datetime.timedelta(minutes=10)
  min20 = datetime.datetime.now() - datetime.timedelta(minutes=20)

  neu_num = result[result['Time']>min10]["Num of tweets"][result['sentiment']==0.0].sum()
  neg_num = result[result['Time']>min10]["Num of tweets"][result['sentiment']==-1.0].sum()
  pos_num = result[result['Time']>min10]["Num of tweets"][result['sentiment']==1.0].sum()

  count_now = df[df['unix'] > min10]['tweet_id'].count()
  count_before = df[ (min20 < df['unix']) & (df['unix'] < min10)]['tweet_id'].count()
  percent = (count_now-count_before)/count_before*100


  # Percentage Number of Tweets changed in Last 10 mins

  # Create the graph 
  children = [
              html.Div([
                  html.Div([
                      dcc.Graph(
                          id='crossfilter-indicator-scatter',
                          figure={
                              'data': [
                                  go.Scatter(
                                      x=time_series,
                                      y=result['Num of tweets'][result['sentiment']==0.0].reset_index(drop=True),
                                      name="Neutrals",
                                      opacity=0.8,
                                      mode='lines',
                                      line=dict(width=0.5, color='rgb(131, 90, 241)'),
                                      stackgroup='one' 
                                  ),
                                  go.Scatter(
                                      x=time_series,
                                      y=result['Num of tweets'][result['sentiment']==-1.0].reset_index(drop=True).apply(lambda x: -x),
                                      name="Negatives",
                                      opacity=0.8,
                                      mode='lines',
                                      line=dict(width=0.5, color='rgb(255, 50, 50)'),
                                      stackgroup='two' 
                                  ),
                                  go.Scatter(
                                      x=time_series,
                                      y=result['Num of tweets'][result['sentiment']==1.0].reset_index(drop=True),
                                      name="Positives",
                                      opacity=0.8,
                                      mode='lines',
                                      line=dict(width=0.5, color='rgb(184, 247, 212)'),
                                      stackgroup='three' 
                                  )
                              ]
                          }
                      )
                  ], style={'width': '73%', 'display': 'inline-block', 'padding': '0 0 0 20'}),
                  
                  html.Div([
                      dcc.Graph(
                          id='pie-chart',
                          figure={
                              'data': [
                                  go.Pie(
                                      labels=['Positives', 'Negatives', 'Neutrals'], 
                                      values=[pos_num, neg_num, neu_num],
                                      name="View Metrics",
                                      marker_colors=['rgba(184, 247, 212, 0.6)','rgba(255, 50, 50, 0.6)','rgba(131, 90, 241, 0.6)'],
                                      textinfo='value',
                                      hole=.65)
                              ],
                              'layout':{
                                  'showlegend':False,
                                  'title':'Tweets In Last 10 Mins',
                                  'annotations':[
                                      dict(
                                          text='{0:.1f}K'.format((pos_num+neg_num+neu_num)/1000),
                                          font=dict(
                                              size=40
                                          ),
                                          showarrow=False
                                      )
                                  ]
                              }

                          }
                      )
                  ], style={'width': '27%', 'display': 'inline-block'})
              ])
              
          ]



  return children


@my_app.callback(Output('live-update-graph-bottom', 'children'),
              [Input('interval-component-slow', 'n_intervals')])
def update_graph_bottom_live(n):
  # Loading data from SQLite
  # DATABASE_URL = os.environ['/content/drive/My Drive/Colab Notebooks/TwitterDash']
  conn = sqlite3.connect('/content/twitter.db')
  c = conn.cursor()
  query = "SELECT tweet_id, text, time_ms, sentiment,polarity, location FROM {}".format(sentiment)
  data = pd.read_sql(query, con=conn)
  conn.close()

  # Convert UTC into PDT
  data['unix'] = pd.to_datetime(data['unix'],unit='ms' )

  # Clean and transform data to enable time series
  result = df.groupby([pd.Grouper(key='unix', freq='10s'), 'sentiment']).count().unstack(fill_value=0).stack().reset_index()
  result = result.rename(columns={"tweet_id": "Num of '{}' mentions".format("tesla"), "unix":"Time"})  
  time_series = result["Time"][result['sentiment']==0.0].reset_index(drop=True)

  # Clean and transform data to enable word frequency
  content = ' '.join(data["tweet"])
  content = re.sub(r"http\S+", "", content)
  content = content.replace('RT ', ' ').replace('&amp;', 'and')
  content = re.sub('[^A-Za-z0-9]+', ' ', content)
  content = content.lower()

  # Filter constants for states in US
  STATES = ['Alabama', 'AL', 'Alaska', 'AK', 'American Samoa', 'AS', 'Arizona', 'AZ', 'Arkansas', 'AR', 'California', 'CA', 'Colorado', 'CO', 'Connecticut', 'CT', 'Delaware', 'DE', 'District of Columbia', 'DC', 'Federated States of Micronesia', 'FM', 'Florida', 'FL', 'Georgia', 'GA', 'Guam', 'GU', 'Hawaii', 'HI', 'Idaho', 'ID', 'Illinois', 'IL', 'Indiana', 'IN', 'Iowa', 'IA', 'Kansas', 'KS', 'Kentucky', 'KY', 'Louisiana', 'LA', 'Maine', 'ME', 'Marshall Islands', 'MH', 'Maryland', 'MD', 'Massachusetts', 'MA', 'Michigan', 'MI', 'Minnesota', 'MN', 'Mississippi', 'MS', 'Missouri', 'MO', 'Montana', 'MT', 'Nebraska', 'NE', 'Nevada', 'NV', 'New Hampshire', 'NH', 'New Jersey', 'NJ', 'New Mexico', 'NM', 'New York', 'NY', 'North Carolina', 'NC', 'North Dakota', 'ND', 'Northern Mariana Islands', 'MP', 'Ohio', 'OH', 'Oklahoma', 'OK', 'Oregon', 'OR', 'Palau', 'PW', 'Pennsylvania', 'PA', 'Puerto Rico', 'PR', 'Rhode Island', 'RI', 'South Carolina', 'SC', 'South Dakota', 'SD', 'Tennessee', 'TN', 'Texas', 'TX', 'Utah', 'UT', 'Vermont', 'VT', 'Virgin Islands', 'VI', 'Virginia', 'VA', 'Washington', 'WA', 'West Virginia', 'WV', 'Wisconsin', 'WI', 'Wyoming', 'WY']
  STATE_DICT = dict(itertools.zip_longest(*[iter(STATES)] * 2, fillvalue=""))
  INV_STATE_DICT = dict((v,k) for k,v in STATE_DICT.items())


  # Clean and transform data to enable geo-distribution
  is_in_US=[]
  geo = data[['Location']]
  df = df.fillna(" ")
  for x in data['Location']:
      check = False
      for s in STATES:
          if s in x:
              is_in_US.append(STATE_DICT[s] if s in STATE_DICT else s)
              check = True
              break
      if not check:
          is_in_US.append(None)

  geo_dist = pd.DataFrame(is_in_US, columns=['State']).dropna().reset_index()
  geo_dist = geo_dist.groupby('State').count().rename(columns={"index": "Number"}).sort_values(by=['Number'], ascending=False).reset_index()
  geo_dist["Log Num"] = geo_dist["Number"].apply(lambda x: math.log(x, 2))

  geo_dist['Full State Name'] = geo_dist['State'].apply(lambda x: INV_STATE_DICT[x])
  geo_dist['text'] = geo_dist['Full State Name'] + '<br>' + 'Num: ' + geo_dist['Number'].astype(str)

  tokenized_word = word_tokenize(content)
  stop_words=set(stopwords.words("english"))
  filtered_sent=[]
  for w in tokenized_word:
    if w not in stop_words:
      filtered_sent.append(w)
  fdist = FreqDist(filtered_sent)
  fd = pd.DataFrame(fdist.most_common(10),columns = ["Word","Frequency"]).drop([0]).reindex()
  fd['Polarity'] = fd['Word'].apply(lambda x: TextBlob(x).sentiment.polarity)
  fd['Marker_Color'] = fd['Polarity'].apply(lambda x: 'rgba(255, 50, 50, 0.6)' if x < -0.1 else \
      ('rgba(184, 247, 212, 0.6)' if x > 0.1 else 'rgba(131, 90, 241, 0.6)'))
  fd['Line_Color'] = fd['Polarity'].apply(lambda x: 'rgba(255, 50, 50, 1)' if x < -0.1 else \
      ('rgba(184, 247, 212, 1)' if x > 0.1 else 'rgba(131, 90, 241, 1)'))
  # Create the graph 
  children = [
            html.Div([
                dcc.Graph(
                    id='x-time-series',
                    figure = {
                        'data':[
                            go.Bar(                          
                                x=fd["Frequency"].loc[::-1],
                                y=fd["Word"].loc[::-1], 
                                name="Neutrals", 
                                orientation='h',
                                marker_color=fd['Marker_Color'].loc[::-1].to_list(),
                                marker=dict(
                                    line=dict(
                                        color=fd['Line_Color'].loc[::-1].to_list(),
                                        width=1),
                                    ),
                            )
                        ],
                        'layout':{
                            'hovermode':"closest"
                        }
                    }        
                )
            ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 0 0 20'}),
            html.Div([
                dcc.Graph(
                    id='y-time-series',
                    figure = {
                        'data':[
                            go.Choropleth(
                                locations=geo_dist['State'], # Spatial coordinates
                                z = geo_dist['Log Num'].astype(float), # Data to be color-coded
                                locationmode = 'USA-states', # set of locations match entries in `locations`
                                #colorscale = "Blues",
                                text=geo_dist['text'], # hover text
                                geo = 'geo',
                                colorbar_title = "Num in Log2",
                                marker_line_color='white',
                                colorscale = ["#fdf7ff", "#835af1"],
                                #autocolorscale=False,
                                #reversescale=True,
                            ) 
                        ],
                        'layout': {
                            'title': "Geographic Segmentation for US",
                            'geo':{'scope':'usa'}
                        }
                    }
                )
            ], style={'display': 'inline-block', 'width': '49%'})
        ]

  return children
    


if __name__ == '__main__':
  my_app.run_server(debug=True)


Overwriting my_app.py


In [8]:
!python my_app.py

Running on http://127.0.0.1:8050/
Debugger PIN: 621-831-844
 * Serving Flask app "my_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 684-868-601
